In [1]:
# utility from fink-science
from fink_science.utilities import concat_col

# user-defined function from the current folder
from processor import rfscore_pca

In [2]:
df = spark.read.format('parquet').load('/media/biswajit/drive/Kilonova_datasets/20191101')

In [3]:
# Required alert columns
what = ['jd', 'fid', 'magpsf', 'sigmapsf']

# Use for creating temp name
prefix = 'c'
what_prefix = [prefix + i for i in what]

# Concatenate historical + current measurements
for colname in what:
    df = concat_col(df, colname, prefix=prefix)
#args = [df.col(i) for i in what_prefix]

In [4]:
df.printSchema()

root
 |-- schemavsn: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- objectId: string (nullable = true)
 |-- candid: long (nullable = true)
 |-- candidate: struct (nullable = true)
 |    |-- jd: double (nullable = true)
 |    |-- fid: integer (nullable = true)
 |    |-- pid: long (nullable = true)
 |    |-- diffmaglim: float (nullable = true)
 |    |-- pdiffimfilename: string (nullable = true)
 |    |-- programpi: string (nullable = true)
 |    |-- programid: integer (nullable = true)
 |    |-- candid: long (nullable = true)
 |    |-- isdiffpos: string (nullable = true)
 |    |-- tblid: long (nullable = true)
 |    |-- nid: integer (nullable = true)
 |    |-- rcid: integer (nullable = true)
 |    |-- field: integer (nullable = true)
 |    |-- xpos: float (nullable = true)
 |    |-- ypos: float (nullable = true)
 |    |-- ra: double (nullable = true)
 |    |-- dec: double (nullable = true)
 |    |-- magpsf: float (nullable = true)
 |    |-- sigmapsf: float (nullab

In [5]:
print(df.select(['objectID']).show())

+------------+
|    objectID|
+------------+
|ZTF19acmcetc|
|ZTF17aaanypg|
|ZTF19acmbxka|
|ZTF19acmbxfe|
|ZTF19acmbtac|
|ZTF18acguzti|
|ZTF19acmbxrc|
|ZTF19acmbwej|
|ZTF18acpvmvh|
|ZTF19acmbvze|
|ZTF19acdtnex|
|ZTF18acgtyfa|
|ZTF18aaacmcm|
|ZTF19acevxhy|
|ZTF19acmbwaw|
|ZTF19acmbxox|
|ZTF19acgjjpl|
|ZTF18aabkgda|
|ZTF17aabtkhd|
|ZTF19acmbwos|
+------------+
only showing top 20 rows

None


In [6]:
import pickle 
pkl_filename = "models/pickle_model.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [7]:
print(rfscore_pca('cjd', 'cfid', 'cmagpsf').values)

Column<b'rfscore_pca(cjd, cfid, cmagpsf)[values]'>


In [8]:

df_change = df.withColumn('pca_score', rfscore_pca('cjd', 'cfid', 'cmagpsf','csigmapsf'))

# print the result for the 20 first alerts
#df_change.select(['objectId', 'cdsxmatch', 'deltamag']).show()

In [9]:
df_change.printSchema()

root
 |-- schemavsn: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- objectId: string (nullable = true)
 |-- candid: long (nullable = true)
 |-- candidate: struct (nullable = true)
 |    |-- jd: double (nullable = true)
 |    |-- fid: integer (nullable = true)
 |    |-- pid: long (nullable = true)
 |    |-- diffmaglim: float (nullable = true)
 |    |-- pdiffimfilename: string (nullable = true)
 |    |-- programpi: string (nullable = true)
 |    |-- programid: integer (nullable = true)
 |    |-- candid: long (nullable = true)
 |    |-- isdiffpos: string (nullable = true)
 |    |-- tblid: long (nullable = true)
 |    |-- nid: integer (nullable = true)
 |    |-- rcid: integer (nullable = true)
 |    |-- field: integer (nullable = true)
 |    |-- xpos: float (nullable = true)
 |    |-- ypos: float (nullable = true)
 |    |-- ra: double (nullable = true)
 |    |-- dec: double (nullable = true)
 |    |-- magpsf: float (nullable = true)
 |    |-- sigmapsf: float (nullab

In [10]:
df_change.select(['pca_score']).show()

+-------------------+
|          pca_score|
+-------------------+
|                0.0|
|0.17748262818840746|
|                0.0|
|                0.0|
|                0.0|
| 0.1940903778117784|
|                0.0|
|                0.0|
| 0.0545446320104014|
|                0.0|
| 0.1909867516543953|
|                0.0|
| 0.1909867516543953|
|0.25635052643671463|
|                0.0|
|                0.0|
|0.19293637671454517|
|0.27040470900787916|
| 0.1951857883239929|
|                0.0|
+-------------------+
only showing top 20 rows

